# Storing data with Spark
Documentation on storing data with spark in general can be found:
 http://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html


In [1]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data, get_dim_data, ROOT_DIR
from pyspark.sql import functions as f
spark = get_spark_session("joins")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

dim_taxi_zones, dim_rates, dim_payments, dim_vendor = get_dim_data(spark)

24/02/17 12:41:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


We have two options now:

1. Overwrite the table completely
2. Append information (use it only with partitioned data!) otherwise results might not be what you expect

In [3]:
yellow_taxi_data.write.mode('overwrite').parquet(ROOT_DIR + "output/yellow_taxi.parquet")

In [4]:
(yellow_taxi_data
 .filter("fare_amount>100")
 .join(dim_vendor, yellow_taxi_data.VendorID == dim_vendor.vendor_id, "full")
 .write
 .mode('overwrite')
 .parquet(ROOT_DIR + "output/yellow_taxi_filtered.parquet"))

# Partitioning and Bucketing

## Partitioning

Partitioning in Apache Spark is a fundamental concept that plays a crucial role in enhancing the performance of distributed data processing. It refers to the division of large datasets into smaller, manageable parts (known as partitions) that can be processed in parallel across different nodes of a Spark cluster. This approach allows Spark to optimize the execution of operations by minimizing data shuffle across the cluster and efficiently utilizing resources.

Effective partitioning is key to achieving high scalability and speed in data processing tasks. Spark provides several ways to control partitioning, including explicit partitioning methods through APIs and automatic partitioning based on the nature of the data and operations being performed. Understanding and optimizing data partitioning can lead to significant improvements in the performance of Spark applications, especially for large-scale data processing tasks.

In [2]:
(yellow_taxi_data
 .write
 .partitionBy("VendorID")
 .mode('overwrite')
 .parquet(ROOT_DIR + "output/yellow_taxi_partitioned.parquet"))

## Bucketing
Bucketing in Apache Spark is an optimization technique that enhances the performance of large-scale data processing, particularly in the context of SQL operations and data storage. Unlike partitioning, which divides data across the cluster based on a partition key, bucketing organizes data into fixed-size buckets based on the value of a hash function applied to a column in your dataset. This approach is especially useful for optimizing join operations and managing data skew.

By bucketing data, Spark can reduce the shuffle required during join operations because it knows in advance which buckets of data are likely to be joined together. This allows for more efficient execution of joins by processing only relevant subsets of data. Additionally, bucketing can improve query performance and data filtering, as operations can be targeted to specific buckets rather than scanning entire partitions.

To implement bucketing in Spark, you define bucket columns and the number of buckets when you write your DataFrame to disk. This upfront organization of data into buckets can significantly speed up subsequent read and join operations on that data. Bucketing is particularly beneficial when working with large datasets where joins and aggregations are common, as it provides a structured way to manage data distribution and optimize query performance.

**Bucketing is not very Dynamic; it can be applied only with Catalogs (Hive/Iceberg/JDBC, etc.) for later querying, and usually, bucketing solves only one local problem. Strongly suggest looking for other solutions before you go to bucketing!**


Bucketing in code:

```python
df.write.bucketBy(3, "id").saveAsTable("bucketed_table")
```